In [1]:
from helpers import *
import numpy as np
import pandas as pd
import os
import pickle
from LDA_helpers import process_tweets
import re
from pre_process_text import clean

In [2]:
DATA_PATH = access_folder()
print(os.listdir(DATA_PATH))

['IRAhandle_tweets_4.csv', 'IRAhandle_tweets_5.csv', 'IRAhandle_tweets_7.csv', 'IRAhandle_tweets_6.csv', 'cleaned_tweets_by_authors.txt', 'IRAhandle_tweets_2.csv', '.DS_Store', 'IRAhandle_tweets_3.csv', 'IRAhandle_tweets_1.csv', 'line_to_author.pickle', '.gitignore', 'hashtags.txt', 'tweets_by_authors.txt', 'IRAhandle_tweets_8.csv', 'IRAhandle_tweets_9.csv']


In [3]:
# loading all data
df_s = []
for file in os.listdir(DATA_PATH):
    if file[-4:] == '.csv':
        df_s.append(pd.read_csv(DATA_PATH + file))
tweets = pd.concat(df_s)
tweets

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer
3,1.674084e+09,GAB1ALDANA,#MyAchillesHeel slippery floors https://t.co/R...,United States,Norwegian,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2160,RETWEET,Hashtager,0,1,HashtagGamer
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer
5,1.674084e+09,GAB1ALDANA,Your opinion on Hillary really matters to a no...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2014,2154,RETWEET,Hashtager,0,1,HashtagGamer
6,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Lilith from Frasier. 0_O,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2156,RETWEET,Hashtager,0,1,HashtagGamer
7,1.674084e+09,GAB1ALDANA,Come find us at the national mall for the #DC ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2157,RETWEET,Hashtager,0,1,HashtagGamer
8,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Trolling celebrities. I have...,United States,English,7/30/2016 20:17,7/30/2016 20:17,3395,2013,2161,RETWEET,Hashtager,0,1,HashtagGamer
9,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Morbid comedy �,United States,English,7/30/2016 20:17,7/30/2016 20:17,3395,2013,2167,RETWEET,Hashtager,0,1,HashtagGamer


In [4]:
# only keeping tweets in English
english_tweets = tweets.loc[tweets['language'] == 'English']
english_tweets.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer
5,1.674084e+09,GAB1ALDANA,Your opinion on Hillary really matters to a no...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2014,2154,RETWEET,Hashtager,0,1,HashtagGamer


In [5]:
# groupby author
txt_path = DATA_PATH + 'tweets_by_authors.txt'
author_group = english_tweets.groupby('author')
line_to_author_dictionary = dict()
index = 0
with open(txt_path, 'w') as f:
    for name, group in author_group:
        line = ''
        for content in group.content.values:
            line = line + str(content)
        f.write(line+'\n')
        line_to_author_dictionary[index] = name
        index += 1
# save dictionary to pickel file
dict_path = DATA_PATH + 'line_to_author.pickle'
save('line_to_author', line_to_author_dictionary, folder_name='data')

In [22]:
hashtag_list = []
for content in english_tweets.content.values:
    hashtags = set([re.sub(r"(\W+)$", "", j) for j in set([i for i in str(content).split() if i.startswith("#")])])
    for hashtag in hashtags:
        if hashtag not in hashtag_list:
            hashtag_list.append(hashtag)

In [24]:
hashtag_path = DATA_PATH + 'hashtags.txt'
with open(hashtag_path, 'w') as f:
    for line in hashtag_list:
        f.write(line+'\n')

In [3]:
txt_path = DATA_PATH + 'tweets_by_authors.txt'
cleaned_txt_path = DATA_PATH + 'cleaned_tweets_by_authors.txt'
f = open(txt_path)
with open(cleaned_txt_path, 'w') as c_f:
    for line in f:
        c_f.write(clean(line)+'\n')
    f.close()